In [1]:
from music21 import converter, instrument, note, chord
import glob
import pickle
import pandas as pd
import tqdm
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

notes = []
log = []

In [33]:
annees=[2002, 2004, 2006, 2008, 2009, 2011, 2013, 2015, 2017, 2018]
notes=[]
for x in annees:
    notesAnnees=pd.read_pickle(r'../data/results/notes_'+str(x))
    notes+=notesAnnees

print(len(notes))

8892971


In [34]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=2,
              batch_size=128, callbacks=callbacks_list)

testing on sample of 1000 notes

In [35]:
notes = notes[0:10000]
# get amount of pitch names
n_vocab = len(set(notes))

In [36]:
#nombre de notes différentes à dispo
n_vocab

2324

In [37]:
#exemples des 10 premieres notes
notes[0:10]

['B-3.F3.D3',
 'B-2.B-1',
 'C#3',
 'C#2',
 'B-3.F3.D3',
 'B-2.B-1',
 'D3.F3.B-2',
 'D3.B-2.F3',
 'F3.D3',
 'B-2']

In [38]:
network_input, network_output = prepare_sequences(notes, n_vocab)



In [39]:
#l'output a pour dimension notes_passées*nb de séquences
network_input.shape

(9900, 100, 1)

In [40]:
#categorique
network_output.shape

(9900, 2324)

In [41]:
model = create_network(network_input, n_vocab)



In [ ]:
train(model, network_input, network_output)



Epoch 1/2
47/78 [=================>............] - ETA: 3:13 - loss: 7.7668